In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path+"/scripts")
    sys.path.append(module_path+"/classes")

review_corpus_path = "../data/tech_review_sent_corpus.pkl"
vocab_path = "../data/glove-tech-revew-vocab.txt"
emb_filename = '../models/w2v_embedding'
aspect_file_path = "../results/aspect.json"
aspect_model_path = "../results/model_param"
vocab_path = "../data/vocab-text-review.txt"

In [1]:
import pandas as pd
import pickle

review_corpus_path = "../data/tech_review_sent_corpus.pkl"

with open(review_corpus_path,"rb") as f:
    tech_review_corpus = pickle.load(f)

print(len(tech_review_corpus))
reviews = pd.DataFrame(tech_review_corpus).review.tolist()

156991


In [3]:
from LDAModeling import tokenize
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=3, max_df = .90, tokenizer = tokenize, ngram_range = (1,2) )
X = vectorizer.fit_transform(reviews)

vocab = { word:i for i, word in enumerate(vectorizer.get_feature_names())}

In [109]:
from gensim.corpora.dictionary import Dictionary
import gensim

corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
id2word = { id:word for word, id in vectorizer.vocabulary_.items()}

dictionary = Dictionary.from_corpus(corpus,id2word=id2word)

In [120]:
from datetime import datetime
from gensim.models import CoherenceModel
import numpy as np

def calc_coherance(topics):
    u_mass = []
    flags = []
    for n, topic in enumerate(topics):
        cm = CoherenceModel(topics=[topic], corpus=corpus, dictionary=dictionary, coherence='u_mass')
        u_mass.append(cm.get_coherence())
        return np.mean(u_mass)

In [134]:
import json
import os

files = [x for x in os.walk("../results/")]

scores = []
for t in files[0][2]:
    if t[:12] == "lda-aspect-k":
        with open(os.path.join("../results",t)) as f:
            aspects = json.load(f)

        topics = [[ word for word, score in words.items() ] for aspect, words in aspects.items() ]

        s = calc_coherance(topics)
        scores.append((t, s))

In [139]:
result = []
for score in scores:
    x = score[0][:-5].split("-")
    result.append({
        "k":x[3],
        "alpha":x[5],
        "beta":x[7],
        "score":score[1]
    })

In [141]:
import pandas as pd

pd.DataFrame(result).to_csv("../data/lda_umass.csv")